In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import os
import json
from pandas.io.json import json_normalize
import cPickle as pk
import matplotlib.pyplot as plt
import matplotlib as mpl
#import seaborn as sns
import datetime as dt
import urllib2
import urllib
from patsy import dmatrices, dmatrix, demo_data

In [3]:
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

In [4]:
handle=open('../data/Kenya.pickle','r')
df2=pk.load(handle)
handle.close()

In [5]:
df2.drop_duplicates(subset=['id','name','funded_date','status','texts.en'],inplace=True)

In [6]:
df2.columns

Index([                     u'activity',                 u'basket_amount',
            u'bonus_credit_eligibility',                     u'borrowers',
                             u'country',                  u'country_code',
       u'currency_exchange_loss_amount',                    u'delinquent',
                         u'description',                 u'funded_amount',
                         u'funded_date',                     u'geo.level',
                           u'geo.pairs',                      u'geo.type',
                                  u'id',                     u'languages',
                        u'lender_count',                   u'loan_amount',
                            u'location',                          u'name',
                                 u'obs',                   u'paid_amount',
                           u'paid_date',                    u'partner_id',
                            u'payments',       u'planned_expiration_date',
                         

In [7]:
index1=range(0,len(df2['name']))

In [8]:
df2.reset_index(index1,inplace=True)

In [9]:
df2a = df2[df2.funded_amount>0]

In [10]:
def mktime(x):
    try:
        tym=dt.datetime.strptime(x,"%Y-%m-%dT%H:%M:%SZ")
        return tym
    except:
        return np.nan

In [11]:
def getyear(x):
    try:
        yr=np.int(x.year)
        return yr
    except:
        return np.nan

In [12]:
fun_dat=df2a['funded_date'].apply(mktime)
df2a.loc[:,'funded_datetime']=fun_dat

/Applications/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:288: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Applications/anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:465: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [13]:
fun_year=df2a['funded_datetime'].apply(getyear)

In [14]:
df2a.loc[:,'funded_year']=fun_year

In [15]:
df2a.columns

Index([                        u'index',                      u'activity',
                       u'basket_amount',      u'bonus_credit_eligibility',
                           u'borrowers',                       u'country',
                        u'country_code', u'currency_exchange_loss_amount',
                          u'delinquent',                   u'description',
                       u'funded_amount',                   u'funded_date',
                           u'geo.level',                     u'geo.pairs',
                            u'geo.type',                            u'id',
                           u'languages',                  u'lender_count',
                         u'loan_amount',                      u'location',
                                u'name',                           u'obs',
                         u'paid_amount',                     u'paid_date',
                          u'partner_id',                      u'payments',
             u'planned_ex

In [16]:
df2b=df2a[(df2a.funded_year.isnull()==False)&(df2a.funded_year!=2016.0)]

In [17]:
df2c=df2b[((df2b.status=='paid')|(df2b.status=='defaulted'))&((df2b.prop_female==0)|(df2b.prop_female==1))&(df2b.num_borrowers==1)]

In [18]:
temp=df2c.prop_female.astype(int)
df2c.loc[:,'prop_female']=temp

In [19]:
df2d=df2c[['status','funded_amount','prop_female','num_borrowers','sector']]

In [20]:
df2d.replace(to_replace='Personal Use',value='Personal_Use', inplace=True)
df2d.status.where(df2d.status=='defaulted',0,inplace=True)
df2d.status.where(df2d.status==0,other=1,inplace=True)

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
/Applications/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:4485: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [21]:
df2d.head()

,status,funded_amount,prop_female,num_borrowers,sector
1,0,1000,0,1,Food
2,0,575,1,1,Clothing
3,0,450,1,1,Food
4,0,500,0,1,Retail
5,0,200,1,1,Agriculture


In [22]:
#funded_amount=df2d.funded_amount
#prop_female=df2d.prop_female
#num_borrowers=df2d.num_borrowers
#sector=df2d.sector


test=dmatrix("funded_amount+sector*prop_female",df2d,return_type="dataframe")
#test.columns

In [23]:
test.columns=['intercept','Arts','Clothing','Construct','Edu','Entertain','Food','Health','Housing','Manufact',
             'Pers_use','Retail','Serv','Transprt','Wholesale','funded_amount','prop_female','ArtsXprop_female',
             'ClothingXprop_female','ConstructXprop_female','EduXprop_female','EntertainXprop_female',
             'FoodXprop_female','HealthXprop_female','HousingXprop_female','ManufactXprop_female','Pers_useXprop_female',
             'RetailXprop_female','ServXprop_female','TranprtXprop_female','WholesaleXprop_female']

In [24]:
test5=test.merge(df2d[['status','num_borrowers']],left_index=True,right_index=True)
test5.drop('num_borrowers',axis=1,inplace=True)

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
from sklearn.cross_validation import KFold
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import train_test_split

In [26]:
iuse, ihold = train_test_split(xrange(test5.shape[0]),stratify=test5.status, train_size=0.85,random_state=123)
mask=np.ones(test5.shape[0], dtype='int')
mask[iuse]=1
mask[ihold]=0
mask = (mask==1)

In [27]:
use=test5[mask]
hold_out=test5[~mask]

In [28]:
use.status.value_counts()

0    49524
1     2539
Name: status, dtype: int64

In [29]:
print test5.shape
print use.shape
print hold_out.shape
print 'number of defaults in hold_out', sum(hold_out.status==1.)
print 'number of defaults in use %i' % sum(use.status==1.)

(61251, 32)
(52063, 32)
(9188, 32)
number of defaults in hold_out 448
number of defaults in use 2539


In [30]:
use.paid=use[use.status==0]

In [31]:
bg,trash=train_test_split(use.paid,train_size=int(round(2539*1.3)),random_state=321)

In [32]:
use2=pd.concat([bg,use[use.status==1]])

In [33]:
use2.iloc[-20:,:-1].columns

Index([u'intercept', u'Arts', u'Clothing', u'Construct', u'Edu', u'Entertain',
       u'Food', u'Health', u'Housing', u'Manufact', u'Pers_use', u'Retail',
       u'Serv', u'Transprt', u'Wholesale', u'funded_amount', u'prop_female',
       u'ArtsXprop_female', u'ClothingXprop_female', u'ConstructXprop_female',
       u'EduXprop_female', u'EntertainXprop_female', u'FoodXprop_female',
       u'HealthXprop_female', u'HousingXprop_female', u'ManufactXprop_female',
       u'Pers_useXprop_female', u'RetailXprop_female', u'ServXprop_female',
       u'TranprtXprop_female', u'WholesaleXprop_female'],
      dtype='object')

In [34]:
logm = LogisticRegression()
scores=cross_validation.cross_val_score(logm,use2.iloc[:,:-1].values,(use2.status==1).values,cv=5,
                                        scoring='roc_auc')
scores.mean()

0.66664302661448183

In [35]:
Xlr, Xtest, ylr, ytest = train_test_split(use2.iloc[:,:-1].values, 
                                          (use2.status==1).values, train_size=0.8, random_state=1234)

In [36]:
clf = LogisticRegression()
clf.fit(Xlr,ylr)
print accuracy_score(clf.predict(Xtest),ytest)

0.642979452055


In [37]:
#the grid of parameters to search over
Cs = [0.001, 0.1, 1, 10, 100]

#your turn                 
for i, c in enumerate(Cs):
    logm = LogisticRegression(C=c,random_state=123)
    scores=cross_validation.cross_val_score(logm,Xlr,ylr,cv=10,
                                        scoring='roc_auc')
    print 'the C value is ',c, ' and the mean AUC-ROC value is ', scores.mean()

the C value is  0.001  and the mean AUC-ROC value is  0.642120790582
the C value is  0.1  and the mean AUC-ROC value is  0.676052534003
the C value is  1  and the mean AUC-ROC value is  0.678857685225
the C value is  10  and the mean AUC-ROC value is  0.678286091628
the C value is  100  and the mean AUC-ROC value is  0.678272077524


In [38]:
from sklearn import grid_search

In [39]:
#  Here is the model, implemented with grid_search
clf = LogisticRegression()
parameters={"C": [0.01, 0.1, 1, 10, 100]}
mod = grid_search.GridSearchCV(clf, param_grid=parameters, cv=5, scoring='roc_auc')
mod.fit(Xlr, ylr)
print "The best value of C is:", mod.best_params_
best=mod.best_estimator_
print "The accuracy score using the best model is:", best.score(Xtest,ytest)

The best value of C is: {'C': 1}
The accuracy score using the best model is: 0.642979452055


In [40]:
from sklearn import metrics

In [41]:
predicted=best.predict(Xtest)
predicted_probs=best.predict_proba(Xtest)
accuracy=best.score(Xtest,ytest)
precision,recall,f1,_= metrics.precision_recall_fscore_support(ytest,predicted,average='binary')
auc = metrics.roc_auc_score(ytest,predicted)
print 'Test accuracy = ', accuracy
print 'Test precision =', precision
print 'Test recall = ', recall
print 'Test f1 = ', f1
print 'Test AUC-ROC = ',auc

Test accuracy =  0.642979452055
Test precision = 0.612813370474
Test recall =  0.441767068273
Test f1 =  0.513418903151
Test AUC-ROC =  0.617152190853


In [42]:
best.classes_

array([False,  True], dtype=bool)